In [27]:
print("hi")

hi


In [28]:
team_codes = [
    "dbacks",    # Arizona Diamondbacks
    "braves",    # Atlanta Braves
    "orioles",   # Baltimore Orioles
    "redsox",    # Boston Red Sox
    "cubs",      # Chicago Cubs
    "whitesox",  # Chicago White Sox
    "reds",      # Cincinnati Reds
    "guardians", # Cleveland Guardians
    "rockies",   # Colorado Rockies
    "tigers",    # Detroit Tigers
    "astros",    # Houston Astros
    "royals",    # Kansas City Royals
    "angels",    # Los Angeles Angels
    "dodgers",   # Los Angeles Dodgers
    "marlins",   # Miami Marlins
    "brewers",   # Milwaukee Brewers
    "twins",     # Minnesota Twins
    "mets",      # New York Mets
    "yankees",   # New York Yankees
    "athletics", # Oakland Athletics
    "phillies",  # Philadelphia Phillies
    "pirates",   # Pittsburgh Pirates
    "padres",    # San Diego Padres
    "giants",    # San Francisco Giants
    "mariners",  # Seattle Mariners
    "cardinals",     # St. Louis Cardinals
    "rays",      # Tampa Bay Rays
    "rangers",   # Texas Rangers
    "bluejays",  # Toronto Blue Jays
    "nationals"  # Washington Nationals
]

print(team_codes)


['dbacks', 'braves', 'orioles', 'redsox', 'cubs', 'whitesox', 'reds', 'guardians', 'rockies', 'tigers', 'astros', 'royals', 'angels', 'dodgers', 'marlins', 'brewers', 'twins', 'mets', 'yankees', 'athletics', 'phillies', 'pirates', 'padres', 'giants', 'mariners', 'cardinals', 'rays', 'rangers', 'bluejays', 'nationals']


In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# Use a raw string to properly handle backslashes in the Windows path
chrome_driver_path = r"C:\Users\caneu\OneDrive\chromedriver-win64\chromedriver.exe"

# Configure Chrome options. Remove '--headless' if you want to see the browser window.
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Create a Service object with your ChromeDriver path
service = Service(chrome_driver_path)

# Initialize the WebDriver with the service and options
driver = webdriver.Chrome(service=service, options=options)



<html lang="en" class="device-desktop no-touchevents pointerevents"><head><script id="twitter-wjs" src="https://platform.twitter.com/widgets.js"></script><script async="" type="text/javascript" src="https://www.googletagservices.com/tag/js/gpt.js"></script><script async="" type="text/javascript" src="//micro.rubiconproject.com/prebid/dynamic/8284.js"></script><script type="text/javascript" async="async" src="https://ans.mlb.com/b/ss/mlbglobal08,mlbyankees/10/JS-2.7.0/s08182141118252?AQB=1&amp;nd


In [41]:
import pandas as pd
from bs4 import BeautifulSoup
import time

# Create an empty list to hold the data for each game block
game_data = []

# Loop over each team code
for team_code in team_codes:
    url = f"https://www.mlb.com/{team_code}/schedule/2025/fullseason"
    print(f"\nProcessing team code: {team_code}")
    print(f"Navigating to URL: {url}")
    
    # Navigate to the target URL
    driver.get(url)
    # Wait a few seconds for the page to load
    time.sleep(10)
    
    # Get the HTML from the driver and parse it with BeautifulSoup
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # Debug: Show a snippet of the HTML content to confirm the page loaded
    #print("HTML snippet (first 500 characters):")
    #print(html[:500])
    
    # Try to find the parent div containing game blocks
    div_element = soup.find("div", class_="list-mode")
    if div_element:
        print("Found parent div with class 'list-mode'.")
        # Optionally, you can print the entire prettified div:
        # print(div_element.prettify())
    else:
        print("Parent div with class 'list-mode' still empty.")
    
    # Find all game blocks
    all_blocks = soup.find_all("div", class_="list-mode-table-wrapper")
    print(f"Found {len(all_blocks)} game blocks for team {team_code}.")
    
    # Loop over each game block and extract the data
    for i, block in enumerate(all_blocks, start=1):
        matchup = block.get("data-tracking-matchup-with-date")
        if not matchup:
            print(f"Block {i}: No matchup attribute found, skipping.")
            continue

        # Debug: print the matchup attribute
        print(f"Block {i}: matchup attribute: {matchup}")

        # Only process if the matchup contains " vs " and does not include "@" 
        if " vs " not in matchup or "@" in matchup:
            print(f"Block {i}: Skipping due to format (missing ' vs ' or contains '@').")
            continue

        # Remove any extra details (e.g., date) by splitting on " on "
        teams_part = matchup.split(" on ")[0]
        teams = teams_part.split(" vs ")

        # Extract game details from the corresponding tags
        date_tag = block.find("div", class_="month-date")
        weekday_tag = block.find("div", class_="weekday")
        time_tag = block.find("div", class_="primary-time")

        game_date = date_tag.get_text(strip=True) if date_tag else None
        day_of_week = weekday_tag.get_text(strip=True) if weekday_tag else None
        game_time = time_tag.get_text(strip=True) if time_tag else None

        # Extract the promotion (if available)
        promo = block.get("data-tracking-featured-promotion")
        promo_text = promo if promo else None

        if len(teams) == 2:
            home_team = teams[0].strip()
            away_team = teams[1].strip()
            print(f"Block {i}: Extracted - Home: {home_team}, Away: {away_team}, "
                  f"Date: {game_date}, Day: {day_of_week}, Time: {game_time}, Promo: {promo_text}")
            
            game_data.append({
                "Home Team": home_team,
                "Away Team": away_team,
                "Game Date": game_date,
                "Day": day_of_week,
                "Game Time": game_time,
                "Promo": promo_text,
            })
        else:
            print(f"Block {i}: Unexpected team format: {matchup}")

# Once all team codes are processed, create a DataFrame
df_games = pd.DataFrame(game_data)
print("\nData extraction complete. DataFrame columns:")
print(df_games.columns)
print("First few rows of the DataFrame:")
print(df_games.head())



Processing team code: dbacks
Navigating to URL: https://www.mlb.com/dbacks/schedule/2025/fullseason?homeGame=true
Found parent div with class 'list-mode'.
Found 163 game blocks for team dbacks.
Block 1: matchup attribute: AZ vs CHC on 3/27
Block 1: Extracted - Home: AZ, Away: CHC, Date: Mar 27, Day: Thu, Time: 7:10 pm MST, Promo: Opening Day
Block 2: matchup attribute: AZ vs CHC on 3/28
Block 2: Extracted - Home: AZ, Away: CHC, Date: Mar 28, Day: Fri, Time: 6:40 pm MST, Promo: Postgame Fireworks
Block 3: matchup attribute: AZ vs CHC on 3/29
Block 3: Extracted - Home: AZ, Away: CHC, Date: Mar 29, Day: Sat, Time: 5:10 pm MST, Promo: [no promotion]
Block 4: matchup attribute: AZ vs CHC on 3/30
Block 4: Extracted - Home: AZ, Away: CHC, Date: Mar 30, Day: Sun, Time: 1:10 pm MST, Promo: Paint at the Park - Special Event
Block 5: matchup attribute: AZ@NYY on 4/1
Block 5: Skipping due to format (missing ' vs ' or contains '@').
Block 6: matchup attribute: AZ@NYY on 4/2
Block 6: Skipping due t

In [43]:
df_games.value_counts("Home Team")

Home Team
ATH    81
LAD    81
TOR    81
TEX    81
STL    81
SF     81
SEA    81
SD     81
PIT    81
PHI    81
NYY    81
NYM    81
MIL    81
MIA    81
LAA    81
KC     81
HOU    81
DET    81
CWS    81
COL    81
CLE    81
CIN    81
BOS    81
BAL    81
AZ     81
WSH    81
ATL    69
CHC    67
MIN    66
TB     64
Name: count, dtype: int64

In [49]:
print(df_games[df_games['Home Team'] == "TB"])

     Home Team Away Team Game Date  Day     Game Time            Promo
2065        TB       COL    Mar 28  Fri   4:10 pm EDT  Schedule Magnet
2066        TB       COL    Mar 29  Sat   4:10 pm EDT   Devil Rays Day
2067        TB       COL    Mar 30  Sun   1:40 pm EDT   [no promotion]
2068        TB       PIT    Mar 31  Mon   7:05 pm EDT   [no promotion]
2069        TB        KC     May 1  Thu   1:10 pm EDT   [no promotion]
...        ...       ...       ...  ...           ...              ...
2124        TB       TOR    Sep 17  Wed   7:05 pm EDT   [no promotion]
2125        TB       TOR    Sep 18  Thu   1:10 pm EDT   [no promotion]
2126        TB       BOS    Sep 19  Fri   7:35 pm EDT     City Connect
2127        TB       BOS    Sep 20  Sat   7:05 pm EDT   Devil Rays Day
2128        TB       BOS    Sep 21  Sun  12:10 pm EDT   [no promotion]

[64 rows x 6 columns]


In [50]:
teams_to_rescrape = ["twins", "cubs", "rays", "braves"]
team_mapping = {
    "twins": "MIN",
    "cubs": "CHC",
    "rays": "TB",    # adjust if your abbreviation is different (e.g. "RAYS")
    "braves": "ATL"
}

# Build a list of the home team abbreviations for these teams.
home_team_abbr = [team_mapping[team] for team in teams_to_rescrape]
print("Removing rows with Home Team in:", home_team_abbr)

# Remove rows where Home Team is one of these problematic teams
df_games_cleaned = df_games[~df_games["Home Team"].isin(home_team_abbr)]
print("Remaining DataFrame shape after removal:", df_games_cleaned.shape)
print("Remaining DataFrame sample:")
print(df_games_cleaned.head())

Removing rows with Home Team in: ['MIN', 'CHC', 'TB', 'ATL']
Remaining DataFrame shape after removal: (2106, 6)
Remaining DataFrame sample:
  Home Team Away Team Game Date  Day    Game Time  \
0        AZ       CHC    Mar 27  Thu  7:10 pm MST   
1        AZ       CHC    Mar 28  Fri  6:40 pm MST   
2        AZ       CHC    Mar 29  Sat  5:10 pm MST   
3        AZ       CHC    Mar 30  Sun  1:10 pm MST   
4        AZ       BAL     Apr 7  Mon  6:40 pm MST   

                               Promo  
0                        Opening Day  
1                 Postgame Fireworks  
2                     [no promotion]  
3  Paint at the Park - Special Event  
4                     [no promotion]  


In [52]:

rescraped_data = []

for team_code in teams_to_rescrape:
    url = f"https://www.mlb.com/{team_code}/schedule/2025/fullseason?homeGame=true"
    print(f"\nScraping data for team '{team_code}' from URL: {url}")
    
    # Navigate to the URL
    driver.get(url)
    
    # Optional: add an extra sleep if you suspect lazy-loading issues
    time.sleep(10)
    
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # Find all game blocks
    all_blocks = soup.find_all("div", class_="list-mode-table-wrapper")
    print(f"Found {len(all_blocks)} game blocks for team '{team_code}'.")
    
    for block in all_blocks:
        # Get the matchup string from the attribute
        matchup = block.get("data-tracking-matchup-with-date")
        if not matchup:
            continue

        # Only continue if the matchup string contains " vs " and does not contain "@" 
        if " vs " not in matchup or "@" in matchup:
            continue

        # Remove any extra details after the team names (e.g., the date)
        teams_part = matchup.split(" on ")[0]
        teams = teams_part.split(" vs ")

        # Extract additional game details from the relevant tags
        date_tag = block.find("div", class_="month-date")
        weekday_tag = block.find("div", class_="weekday")
        time_tag = block.find("div", class_="primary-time")

        game_date = date_tag.get_text(strip=True) if date_tag else None
        day_of_week = weekday_tag.get_text(strip=True) if weekday_tag else None
        game_time = time_tag.get_text(strip=True) if time_tag else None

        # Extract promotion info from the attribute (if available)
        promo = block.get("data-tracking-featured-promotion")
        promo_text = promo if promo else None

        # Debugging output: show what is being extracted
        print("Matchup:", matchup)
        print("Extracted - Date:", game_date, "| Day:", day_of_week, "| Time:", game_time, "| Promo:", promo_text)

        # Ensure we have exactly two teams in the matchup string
        if len(teams) == 2:
            home_team = teams[0].strip()
            away_team = teams[1].strip()

            rescraped_data.append({
                "Home Team": home_team,
                "Away Team": away_team,
                "Game Date": game_date,
                "Day": day_of_week,
                "Game Time": game_time,
                "Promo": promo_text,
            })
        else:
            print("Unexpected team format for team", team_code, ":", matchup)

# Create a DataFrame for the rescraped data
df_rescraped = pd.DataFrame(rescraped_data)
print("\nRescraped DataFrame shape:", df_rescraped.shape)
print("Rescraped DataFrame sample:")
print(df_rescraped.head())


Scraping data for team 'twins' from URL: https://www.mlb.com/twins/schedule/2025/fullseason?homeGame=true
Found 163 game blocks for team 'twins'.
Matchup: MIN vs HOU on 4/3
Extracted - Date: Apr 3 | Day: Thu | Time: 3:10 pm CDT | Promo: Opening Day Beanie
Matchup: MIN vs HOU on 4/5
Extracted - Date: Apr 5 | Day: Sat | Time: 1:10 pm CDT | Promo: [no promotion]
Matchup: MIN vs HOU on 4/6
Extracted - Date: Apr 6 | Day: Sun | Time: 1:10 pm CDT | Promo: Kids Opening Day Beanie
Matchup: MIN vs DET on 4/11
Extracted - Date: Apr 11 | Day: Fri | Time: 7:10 pm CDT | Promo: [no promotion]
Matchup: MIN vs DET on 4/12
Extracted - Date: Apr 12 | Day: Sat | Time: 1:10 pm CDT | Promo: [no promotion]
Matchup: MIN vs DET on 4/13
Extracted - Date: Apr 13 | Day: Sun | Time: 1:10 pm CDT | Promo: Kids Day
Matchup: MIN vs NYM on 4/14
Extracted - Date: Apr 14 | Day: Mon | Time: 6:40 pm CDT | Promo: Military Discount
Matchup: MIN vs NYM on 4/15
Extracted - Date: Apr 15 | Day: Tue | Time: 6:40 pm CDT | Promo: 

In [53]:
df_games_cleaned

,Home Team,Away Team,Game Date,Day,Game Time,Promo
0,AZ,CHC,Mar 27,Thu,7:10 pm MST,Opening Day
1,AZ,CHC,Mar 28,Fri,6:40 pm MST,Postgame Fireworks
2,AZ,CHC,Mar 29,Sat,5:10 pm MST,[no promotion]
3,AZ,CHC,Mar 30,Sun,1:10 pm MST,Paint at the Park - Special Event
4,AZ,BAL,Apr 7,Mon,6:40 pm MST,[no promotion]
...,...,...,...,...,...,...
2367,WSH,ATL,Sep 16,Tue,6:45 pm EDT,[no promotion]
2368,WSH,ATL,Sep 17,Wed,4:05 pm EDT,[no promotion]
2369,WSH,CWS,Sep 26,Fri,6:45 pm EDT,Nats Postgame Summer Concert Series: Riley Green
2370,WSH,CWS,Sep 27,Sat,4:05 pm EDT,[no promotion]


In [56]:
final_df = pd.concat([df_games_cleaned, df_rescraped])


In [61]:
team_mapping = {
    'AZ': 'Arizona Diamondbacks',
    'BAL': 'Baltimore Orioles',
    'BOS': 'Boston Red Sox',
    'CWS': 'Chicago White Sox',
    'CIN': 'Cincinnati Reds',
    'CLE': 'Cleveland Guardians',  # or Indians if you prefer historical names
    'COL': 'Colorado Rockies',
    'DET': 'Detroit Tigers',
    'HOU': 'Houston Astros',
    'KC': 'Kansas City Royals',
    'LAA': 'Los Angeles Angels',
    'LAD': 'Los Angeles Dodgers',
    'MIA': 'Miami Marlins',
    'MIL': 'Milwaukee Brewers',
    'NYM': 'New York Mets',
    'NYY': 'New York Yankees',
    'ATH': 'Athletics',  # Assuming "ATH" stands for Athletics
    'PHI': 'Philadelphia Phillies',
    'PIT': 'Pittsburgh Pirates',
    'SD': 'San Diego Padres',
    'SF': 'San Francisco Giants',
    'SEA': 'Seattle Mariners',
    'STL': 'St. Louis Cardinals',
    'TEX': 'Texas Rangers',
    'TOR': 'Toronto Blue Jays',
    'WSH': 'Washington Nationals',
    'MIN': 'Minnesota Twins',
    'CHC': 'Chicago Cubs',
    'TB': 'Tampa Bay Rays',
    'ATL': 'Atlanta Braves'
}

print("Mapping short codes to full names:")
for code, full in team_mapping.items():
    print(f"  {code} -> {full}")

# Update the DataFrame's "Home Team" and "Away Team" columns
final_df['Home Team'] = final_df['Home Team'].map(team_mapping)
final_df['Away Team'] = final_df['Away Team'].map(team_mapping)

Mapping short codes to full names:
  AZ -> Arizona Diamondbacks
  BAL -> Baltimore Orioles
  BOS -> Boston Red Sox
  CWS -> Chicago White Sox
  CIN -> Cincinnati Reds
  CLE -> Cleveland Guardians
  COL -> Colorado Rockies
  DET -> Detroit Tigers
  HOU -> Houston Astros
  KC -> Kansas City Royals
  LAA -> Los Angeles Angels
  LAD -> Los Angeles Dodgers
  MIA -> Miami Marlins
  MIL -> Milwaukee Brewers
  NYM -> New York Mets
  NYY -> New York Yankees
  ATH -> Athletics
  PHI -> Philadelphia Phillies
  PIT -> Pittsburgh Pirates
  SD -> San Diego Padres
  SF -> San Francisco Giants
  SEA -> Seattle Mariners
  STL -> St. Louis Cardinals
  TEX -> Texas Rangers
  TOR -> Toronto Blue Jays
  WSH -> Washington Nationals
  MIN -> Minnesota Twins
  CHC -> Chicago Cubs
  TB -> Tampa Bay Rays
  ATL -> Atlanta Braves


In [62]:
final_df

,Home Team,Away Team,Game Date,Day,Game Time,Promo
0,Arizona Diamondbacks,Chicago Cubs,Mar 27,Thu,7:10 pm MST,Opening Day
1,Arizona Diamondbacks,Chicago Cubs,Mar 28,Fri,6:40 pm MST,Postgame Fireworks
2,Arizona Diamondbacks,Chicago Cubs,Mar 29,Sat,5:10 pm MST,[no promotion]
3,Arizona Diamondbacks,Chicago Cubs,Mar 30,Sun,1:10 pm MST,Paint at the Park - Special Event
4,Arizona Diamondbacks,Baltimore Orioles,Apr 7,Mon,6:40 pm MST,[no promotion]
...,...,...,...,...,...,...
319,Atlanta Braves,Washington Nationals,Sep 23,Tue,7:15 pm EDT,[no promotion]
320,Atlanta Braves,Washington Nationals,Sep 24,Wed,12:15 pm EDT,[no promotion]
321,Atlanta Braves,Pittsburgh Pirates,Sep 26,Fri,7:15 pm EDT,[no promotion]
322,Atlanta Braves,Pittsburgh Pirates,Sep 27,Sat,7:15 pm EDT,[no promotion]


In [59]:
final_df.to_csv("mlb_games_info.csv")

In [63]:
driver.quit()

In [64]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# Use a raw string to properly handle backslashes in the Windows path
chrome_driver_path = r"C:\Users\caneu\OneDrive\chromedriver-win64\chromedriver.exe"

# Configure Chrome options. Remove '--headless' if you want to see the browser window.
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Create a Service object with your ChromeDriver path
service = Service(chrome_driver_path)

# Initialize the WebDriver with the service and options
driver = webdriver.Chrome(service=service, options=options)

url = f"https://www.mlb.com/team"
print(f"Navigating to URL: {url}")

# Navigate to the target URL
driver.get(url)
# Wait a few seconds for the page to load
time.sleep(10)

# Get the HTML from the driver and parse it with BeautifulSoup
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Debug: Show a snippet of th

Navigating to URL: https://www.mlb.com/team


In [66]:
team_boxes = soup.find_all("div", class_="p-forge-list-item l-grid__content l-grid__content--card u-app-hide")

In [71]:
team_boxes[1].get('id')

'athletics'

In [69]:
team_venue_info = {}
for t in team_boxes:
    
    
    p_tag = t.find("p")

    # Extract text elements, ignoring links
    text_elements = [text.strip() for text in p_tag.find_all(string=True) if text.strip()]

    venue_name = text_elements[0] 
    full_address = text_elements[1]
    pattern = r"^(.*)\n([\w\s]+),\s([A-Z]{2})\s(\d{5})$"

    match = re.search(pattern, full_address)

    address = match.group(1)
    city = match.group(2)
    state = match.group(3)
    postal_code = match.group(4)
    team_name = t.get('id')

    


Extracted Text:
Chase Field
401 East Jefferson Street
Phoenix, AZ 85004
Phone: (602) 462-6500
dbacks.com
losdbacks.com
Extracted Text:
Sutter Health Park
400 Ballpark Drive
West Sacramento, CA 95691
Phone: (916) 371-HITS (4487)
athletics.com
losatleticos.com
Extracted Text:
Truist Park
755 Battery Avenue
Atlanta, GA 30339
Phone: (404) 614-2300
braves.com
losbravosbeisbol.com
Extracted Text:
Oriole Park
333 West Camden Street
Baltimore, MD 21201
Phone: (410) 685-9800
orioles.com
Extracted Text:
Fenway Park
4 Jersey Street
Boston, MA 02215
Phone: (617) 267-9440
redsox.com
losredsox.com
Extracted Text:
Wrigley Field
1060 West Addison
Chicago, IL 60613-4397
Phone: (773) 404-2827
cubs.com
loscubs.com
Extracted Text:
Rate Field
333 West 35th Street
Chicago, IL 60616
Phone: (312) 674-1000
whitesox.com
loswhitesox.com
Extracted Text:
Great American Ball Park
100 Main Street
Cincinnati, OH 45202-4109
Phone: (513) 765-7000
reds.com
Extracted Text:
Progressive Field
2401 Ontario Street
Cleveland,

In [72]:
import re

text = """401 East Jefferson Street
Phoenix, AZ 85004"""

# Regular expression pattern to extract address, city, state, and ZIP code
pattern = r"^(.*)\n([\w\s]+),\s([A-Z]{2})\s(\d{5})$"

match = re.search(pattern, text)

In [73]:
match

<re.Match object; span=(0, 43), match='401 East Jefferson Street\nPhoenix, AZ 85004'>